# Experimentation Notebook

In this notebook we complete the experimentation process data scientists would preform to create a model. The [dataset](../datasets/covertype/README.md) used is based on **Covertype Data Set** from UCI Machine Learning Repository.

The training code uses `scikit-learn` for data pre-processing and modeling.

## Imports

In [ ]:
import json
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import seaborn as sns
import matplotlib.pyplot as plt

from jinja2 import Template
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

## Data Import
This demo uses data from an open source database which is stored in a public bucket.  

In [ ]:
DATA_ROOT = 'gs://workshop-datasets/covertype'
TRAINING_FILE_PATH = DATA_ROOT + '/training/dataset.csv'
VALIDATION_FILE_PATH = DATA_ROOT + '/evaluation/dataset.csv'

In [ ]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)

## Explore the Covertype dataset 

In [ ]:
df_train

In [ ]:
print(df_train.shape)
print(df_validation.shape)

### Visualize Data 

Selected plots have been created to illustrate the data exploration and analysis process

In [ ]:
sns.violinplot(data=df_train,x='Cover_Type', y='Elevation')  

In [ ]:
sns.violinplot(data=df_train,x='Cover_Type', y='Wilderness_Area')

## Develop the training application

### Configure the `sklearn` training pipeline.

The training pipeline preprocesses data by standardizing all numeric features using `sklearn.preprocessing.StandardScaler` and encoding all categorical features using `sklearn.preprocessing.OneHotEncoder`. It uses stochastic gradient descent linear classifier (`SGDClassifier`) for modeling.

In [ ]:
numeric_feature_indexes = slice(0, 10)
categorical_feature_indexes = slice(10, 12)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log'))
])

### Convert all numeric features to `float64`

To avoid warning messages from `StandardScaler` all numeric features are converted to `float64`.

In [ ]:
num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}

df_train = df_train.astype(num_features_type_map)
df_validation = df_validation.astype(num_features_type_map)

### Run the pipeline locally.

In [ ]:
X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=200)
pipeline.fit(X_train, y_train)

### Calculate the trained model's accuracy.

In [ ]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

### Next: Productionising the model

Now that we are happy with the models performance, we want to productionise the model